rationale, results, limitations, and recommendations.

## Business Understanding
1. Business problem
2. Understand the objectives 
3. Key stakeholders 
4. Context for the solution to be implemented 



### Business problem 
The business seeks to understand public sentiment regarding Apple and Google products based on social media discussions. Analyzing sentiment on platforms like Twitter can provide actionable insights into customer preferences, satisfaction, and concerns. To address this, an NLP model must be developed to automatically classify the sentiment (positive, negative, or neutral) in Tweets mentioning these products. The model aims to streamline sentiment analysis, reducing manual effort, and enabling timely responses to market trends and customer feedback, ultimately improving product marketing and customer engagement strategies.

### Objectives 
1. To develop a model to classify tweets as positive, negative, or neutral regardingg Apple and Google products.
2. To optimize the model to achieve high accuracy and precision in sentiment predictions.
3. To ensure the model can be adapted for sentiment analysis of other brands or products in the future.

### Key stakeholders
1. Data Scientists
2. Product Managers
3. Marketing teams
4. Executive and Decision Makers

### Data understanding
The data being used in this project consists of over 9,000 Tweets about Apple and Google products, which have been labeled by human raters as positive, negative, or neutral. This dataset is well-suited for the business problem because it reflects real user opinions and interactions on social media, providing valuable insight into how customers feel about these products. The labeled sentiment in each tweet offers a solid foundation for building and training a model to automatically classify sentiment, aligning with the goal of improving sentiment analysis automation.

The context for implementing the solution involves leveraging sentiment analysis to gain insights into public perception of Apple and Google products through Twitter data. The model will analyze tweets rated as positive, negative, or neutral, providing stakeholders with valuable information on consumer attitudes. This can inform marketing strategies, product improvements, and customer engagement initiatives, helping the companies understand their brand reputation and customer satisfaction levels. The insights derived from the model will aid in decision-making processes and strategic planning.


In [6]:
#libraries and packages to use

import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt', quiet=True)
np.random.seed(0)

In [10]:
#load the data
sentiments = pd.read_csv("tweet_product_company.csv")
sentiments

FileNotFoundError: [Errno 2] No such file or directory: 'tweet_product_company.csv'

## Data preparation
 why did you choose the data preparation steps that you did, and what was the result?
  such as feature engineering, using pipelines, or using unsupervised techniques
  Goes above and beyond with data preparation, such as feature engineering, using pipelines, or using unsupervised techniques

Supervised learning is the core of this project, but feel free to use unsupervised techniques for data analysis or preparation

## Modeling
what modeling package(s) did you use, which model(s) within the package(s), and what tuning steps did you take?
Goes above and beyond in the modeling process, such as using models from multiple different packages and different model explainability tools

## Evaluation

how well did your final model perform?